In [4]:
import sys
import logging
import re
import psycopg2
import requests
import pandas as pd
from fredapi import Fred
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from docopt import docopt

from pprint import pprint as pp


__version__ = '0.0.1'


def get_db_engine(db_name, user, password, host, port=5432):

    db_connection = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        database=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )

    engine = create_engine(db_connection)
    if not database_exists(engine.url):
        try:
            create_database(engine.url)

        except Exception as exc:
            logger.exception('Could not create database. \nException: %r', exc)
            return exc
    return engine


def main():

    user = 'donal'
    password = 'donal'
    host = 'localhost'
    api_key = '01af77900eb060649a7c504ee0705b4d'
    db_name = 'fred_db'
    port = 5432

    engine = get_db_engine(db_name, user, password, host, port)

    fred = Fred(api_key=api_key)
    gdp = fred.get_series('GDPC1').rename('gdp')  # ('GDP')
    um_cust_sent_index = fred.get_series('UMCSENT').rename('umcsent')  # ('UM customer sentiment index')
    us_civ_unemploy_rate = fred.get_series('UNRATE').rename('unrate')  # ('US  Civilian Unemployment Rate')

    frames = [gdp, um_cust_sent_index, us_civ_unemploy_rate]
    fred_data = pd.concat(frames, axis=1)

    return fred_data

if __name__ == "__main__":
    data = main()

In [13]:
from datetime import datetime, date
d = data.index.values[0]
print(type(d), '   ', d)
d = datetime.utcfromtimestamp(d.astype(int))
print(type(d), '   ', d)

<class 'numpy.datetime64'>     1947-01-01T00:00:00.000000000


OSError: [Errno 75] Value too large for defined data type